In [3]:
import pandas as pd
import geopandas as gpd
import numpy as np
import os
from sklearn.neighbors import KDTree

from scipy.spatial import KDTree 
from shapely.geometry import Point

DATA_DIR = '../data/raw/'
INTERIM_DIR = '../data/interim/' 
PROCESSED_DIR = '../data/processed/' 

In [4]:
df_pin = pd.read_csv('../data/interim/universe_pin.csv')
df_pin['pin10'] = df_pin['pin10'].astype(int)
df_pin

,pin10,lon,lat
0,2503106015,-87.623728,41.733446
1,2510117011,-87.616102,41.717261
2,2022402038,-87.610924,41.771698
3,2034101010,-87.622505,41.750465
4,2502207033,-87.587231,41.735210
...,...,...,...
157602,7293060130,NaN,NaN
157603,1329403018,-87.771926,41.929859
157604,2102260010,NaN,NaN
157605,3120300007,-87.768431,41.500254


In [5]:
df_sales = pd.read_csv(os.path.join(DATA_DIR, 'sales_data.csv'))
df_sales

/var/folders/fk/9chv4jr11ln50p4dhsxntpcw0000gn/T/ipykernel_58215/3652251578.py:1: DtypeWarning: Columns (8,10,15) have mixed types. Specify dtype option on import or set low_memory=False.
  df_sales = pd.read_csv(os.path.join(DATA_DIR, 'sales_data.csv'))


,pin,year,township_code,neighborhood_code,class,sale_date,is_mydec_date,sale_price,sale_document_num,sale_deed_type,mydec_deed_type,sale_seller_name,is_multisale,num_parcels_sale,sale_buyer_name,sale_type,sale_filter_same_sale_within_365,sale_filter_less_than_10k,sale_filter_deed_type,row_id
0,25031060150000,2024,70,70111,234,"January 16, 2024",True,"$260,000",2402413181,Warranty,Warranty Deed,CHRISTOPHER SHAW,False,1,CALVIN GRANDBERRY,NaN,False,False,False,7573637
1,25101170110000,2024,70,70111,203,"January 25, 2024",True,"$299,900",2402913199,Trustee,Trustee Deed,9731 FOREST LAND TRUST,False,1,KIMBERLY J ADAMS,NaN,False,False,False,7573642
2,20224020380000,2024,70,70030,211,"January 04, 2024",True,"$385,000",2402246046,Warranty,Warranty Deed,ANNA COUNTS,False,1,JOSHUA ISHMAEL HERNANDEZ,NaN,False,False,False,7573651
3,20341010100000,2024,70,70111,203,"January 11, 2024",True,"$237,000",2401813310,Warranty,Warranty Deed,"NCRC HOUSING REHAB FUND, LLC",False,1,TRYNELL WILLIAMS,NaN,False,False,False,7573654
4,25022070330000,2024,70,70080,241,"January 10, 2024",True,"$455,000",2402306427,Warranty,NaN,HAROLD COLLINS,True,2,NaN,NaN,False,False,False,7573660
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2615947,27034000541111,2024,28,28031,299,"December 02, 2024",True,"$185,000",2435502119,Warranty,NaN,ROBERT B. FICEK,True,2,GAYLINN M. VARKALIS,NaN,False,False,False,7693745
2615948,27214050741034,2024,28,28099,299,"December 16, 2024",True,"$291,500",2435218268,Executor,NaN,NaN,True,2,NaN,NaN,False,False,True,7693746
2615949,27214050741016,2024,28,28099,299,"December 16, 2024",True,"$291,500",2435218268,Executor,NaN,NaN,True,2,NaN,NaN,False,False,True,7693747
2615950,27022090100000,2024,28,28031,278,"November 11, 2024",True,"$530,000",2435202213,Trustee,Trustee Deed,THOMAS BOBERAND DONNA BOBER AS TRUSTEES UNDER ...,False,1,KAMALIDEEN O ABDUR-RAHMAN,NaN,False,False,False,7693748


In [6]:
df_sales['pin10'] = df_sales['pin'].astype(str).str[:10].astype(int)
df_sales.dtypes

pin                                  int64
year                                 int64
township_code                        int64
neighborhood_code                    int64
class                               object
sale_date                           object
is_mydec_date                         bool
sale_price                          object
sale_document_num                   object
sale_deed_type                      object
mydec_deed_type                     object
sale_seller_name                    object
is_multisale                          bool
num_parcels_sale                     int64
sale_buyer_name                     object
sale_type                           object
sale_filter_same_sale_within_365      bool
sale_filter_less_than_10k             bool
sale_filter_deed_type                 bool
row_id                               int64
pin10                                int64
dtype: object

In [7]:
df_merged = pd.merge(df_sales, df_pin, on='pin10', how='left')
df_merged

,pin,year,township_code,neighborhood_code,class,sale_date,is_mydec_date,sale_price,sale_document_num,sale_deed_type,...,num_parcels_sale,sale_buyer_name,sale_type,sale_filter_same_sale_within_365,sale_filter_less_than_10k,sale_filter_deed_type,row_id,pin10,lon,lat
0,25031060150000,2024,70,70111,234,"January 16, 2024",True,"$260,000",2402413181,Warranty,...,1,CALVIN GRANDBERRY,NaN,False,False,False,7573637,2503106015,-87.623728,41.733446
1,25101170110000,2024,70,70111,203,"January 25, 2024",True,"$299,900",2402913199,Trustee,...,1,KIMBERLY J ADAMS,NaN,False,False,False,7573642,2510117011,-87.616102,41.717261
2,20224020380000,2024,70,70030,211,"January 04, 2024",True,"$385,000",2402246046,Warranty,...,1,JOSHUA ISHMAEL HERNANDEZ,NaN,False,False,False,7573651,2022402038,-87.610924,41.771698
3,20341010100000,2024,70,70111,203,"January 11, 2024",True,"$237,000",2401813310,Warranty,...,1,TRYNELL WILLIAMS,NaN,False,False,False,7573654,2034101010,-87.622505,41.750465
4,25022070330000,2024,70,70080,241,"January 10, 2024",True,"$455,000",2402306427,Warranty,...,2,NaN,NaN,False,False,False,7573660,2502207033,-87.587231,41.735210
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2615947,27034000541111,2024,28,28031,299,"December 02, 2024",True,"$185,000",2435502119,Warranty,...,2,GAYLINN M. VARKALIS,NaN,False,False,False,7693745,2703400054,-87.840266,41.636986
2615948,27214050741034,2024,28,28099,299,"December 16, 2024",True,"$291,500",2435218268,Executor,...,2,NaN,NaN,False,False,True,7693746,2721405074,-87.855523,41.592118
2615949,27214050741016,2024,28,28099,299,"December 16, 2024",True,"$291,500",2435218268,Executor,...,2,NaN,NaN,False,False,True,7693747,2721405074,-87.855523,41.592118
2615950,27022090100000,2024,28,28031,278,"November 11, 2024",True,"$530,000",2435202213,Trustee,...,1,KAMALIDEEN O ABDUR-RAHMAN,NaN,False,False,False,7693748,2702209010,NaN,NaN


In [8]:
# Drop all rows with missing lon/lat
df_merged = df_merged.dropna(subset=['lon', 'lat'])
df_merged

,pin,year,township_code,neighborhood_code,class,sale_date,is_mydec_date,sale_price,sale_document_num,sale_deed_type,...,num_parcels_sale,sale_buyer_name,sale_type,sale_filter_same_sale_within_365,sale_filter_less_than_10k,sale_filter_deed_type,row_id,pin10,lon,lat
0,25031060150000,2024,70,70111,234,"January 16, 2024",True,"$260,000",2402413181,Warranty,...,1,CALVIN GRANDBERRY,NaN,False,False,False,7573637,2503106015,-87.623728,41.733446
1,25101170110000,2024,70,70111,203,"January 25, 2024",True,"$299,900",2402913199,Trustee,...,1,KIMBERLY J ADAMS,NaN,False,False,False,7573642,2510117011,-87.616102,41.717261
2,20224020380000,2024,70,70030,211,"January 04, 2024",True,"$385,000",2402246046,Warranty,...,1,JOSHUA ISHMAEL HERNANDEZ,NaN,False,False,False,7573651,2022402038,-87.610924,41.771698
3,20341010100000,2024,70,70111,203,"January 11, 2024",True,"$237,000",2401813310,Warranty,...,1,TRYNELL WILLIAMS,NaN,False,False,False,7573654,2034101010,-87.622505,41.750465
4,25022070330000,2024,70,70080,241,"January 10, 2024",True,"$455,000",2402306427,Warranty,...,2,NaN,NaN,False,False,False,7573660,2502207033,-87.587231,41.735210
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2615944,15261030070000,2024,34,34010,212,"September 10, 2024",True,"$499,999",2435223226,Trustee,...,2,NaN,NaN,False,False,False,7693689,1526103007,-87.838232,41.849476
2615946,27034000541063,2024,28,28031,299,"December 02, 2024",True,"$185,000",2435502119,Warranty,...,2,GAYLINN M. VARKALIS,NaN,False,False,False,7693744,2703400054,-87.840266,41.636986
2615947,27034000541111,2024,28,28031,299,"December 02, 2024",True,"$185,000",2435502119,Warranty,...,2,GAYLINN M. VARKALIS,NaN,False,False,False,7693745,2703400054,-87.840266,41.636986
2615948,27214050741034,2024,28,28099,299,"December 16, 2024",True,"$291,500",2435218268,Executor,...,2,NaN,NaN,False,False,True,7693746,2721405074,-87.855523,41.592118


In [9]:
CTA_FILE_PATH = os.path.join(DATA_DIR, 'cta_l_stops.geojson')
gdf_cta = gpd.read_file(CTA_FILE_PATH)
gdf_cta.to_crs(epsg=4326, inplace=True)
gdf_cta.head()

,station_id,point_y,pknrd,legend,point_x,address,ada,longname,lines,geometry
0,970,1896352.78926296,False,Blue Line,1144440.97667316,720 S. Cicero Avenue,False,Cicero-Congress,Blue Line (Congress),POINT (-87.74517 41.87161)
1,20,1901803.39560403,False,Green Line,1128608.76033842,1 S. Harlem Avenue,True,Harlem-Lake,Green Line (Lake),POINT (-87.80318 41.88685)
2,610,1901950.19138017,False,Green Line,1133921.86037537,36 N. Ridgeland Avenue,False,Ridgeland,Green Line (Lake),POINT (-87.78366 41.88716)
3,230,1937256.04587705,True,Blue Line,1118914.13069739,5800 N. Cumberland Avenue,True,Cumberland,Blue Line,POINT (-87.83803 41.98429)
4,1700,1900828.68763513,False,Multiple Lines,1176812.47724731,29 N. Wabash,True,Washington/Wabash,"Brown, Orange, Pink, Purple (Express), Green",POINT (-87.62619 41.88322)


In [10]:
# Create Point objects from lat/lon columns
geometry = [Point(xy) for xy in zip(df_merged['lon'], df_merged['lat'])]

In [11]:
# Create a GeoDataFrame for sales data
gdf_properties = gpd.GeoDataFrame(
    df_merged,
    geometry=geometry,
    crs="EPSG:4326"
)
print(f"Properties converted to GeoDataFrame. Shape: {gdf_properties.shape}")

Properties converted to GeoDataFrame. Shape: (877037, 24)


In [12]:
# Convert CTA coordinates (Lon/Lat) to a 2D array
cta_coordinates = np.array(list(zip(gdf_cta.geometry.x, gdf_cta.geometry.y)))
# Convert property coordinates to a 2D array
property_coordinates = np.array(list(zip(gdf_properties.geometry.x, gdf_properties.geometry.y)))

In [13]:
# KDTree for fast nearest neighbor searching
tree = KDTree(cta_coordinates)
# Query the KDTree
distances, indices = tree.query(property_coordinates, k=1)

In [14]:
gdf_properties['min_distance_deg'] = distances
nearest_stations = gdf_cta.iloc[indices].reset_index(drop=True)
gdf_properties['nearest_cta_stop'] = nearest_stations['longname']
gdf_properties['nearest_cta_lines'] = nearest_stations['lines']

In [15]:
M_PER_DEGREE = 111111 
gdf_properties['min_distance_meters'] = gdf_properties['min_distance_deg'] * M_PER_DEGREE
gdf_properties[['pin', 'sale_price', 'min_distance_meters', 'nearest_cta_stop', 'nearest_cta_lines']].head()

,pin,sale_price,min_distance_meters,nearest_cta_stop,nearest_cta_lines
0,25031060150000,"$260,000",242.302900,87th,Red Line
1,25101170110000,"$299,900",1084.531512,95th/Dan Ryan,Red Line
2,20224020380000,"$385,000",1068.335623,King Drive,Green Line
3,20341010100000,"$237,000",297.112480,79th,Red Line
4,25022070330000,"$455,000",4169.116857,87th,Red Line


In [16]:
df_final = pd.DataFrame(gdf_properties.drop(columns=['geometry', 'min_distance_deg']))
df_final.to_csv(os.path.join(INTERIM_DIR, 'sales_data_enriched.csv'), index=False)